### Bob möchte nun die Satoshis an ein spezielles von ihm erstelltes Skript binden. 
Das Skript ist der Satz: **"We make it visible"**. Auch dazu eignet sich die **P2SH** Transaktion.

### Aufgrund der Tatsache, dass wir mit einer "leeren" Blockchain beginnen, müssen zunächst ein paar Vorarbeiten geleistet werden.
    1. Verbindung zur Blockchain in regtest Modus aufbauen
    2. Anlegen zweier Nodes und Wallets (Alice und Bob)
    3. Mining 101 Blöcke damit Alice Wallet Coins enthält
    4. Funding der Transaktion welche später als Input für die P2SH-Locking Transaktion genutzt wird.
    5. Bob erzeugt das von ihm gewüsnchte Locking Skript

In [1]:
import sys
sys.path.insert(0, "/media/henning/Volume/Programming/Bitcoin/bitcoin/test/functional")
from test_framework.test_shell import TestShell
from io import BytesIO
from decimal import Decimal

from src.transaction import CTx, CTxIn, CTxOut
from src.script import Script, p2sh_script, p2pk_script
from src.ecdsa import PrivateKey
from src.crypto import hash160
from src.helper import string_to_hex

In [2]:
test = TestShell().setup(num_nodes=2, setup_clean_chain=True)
alice = test.nodes[0]
bob = test.nodes[1]

alice.createwallet("alice", descriptors=False)
bob.createwallet("bob", descriptors=False)

alice_address_0 = alice.getnewaddress("", "legacy")
test.generatetoaddress(alice, 101, alice_address_0)

alice_address_1 = alice.getnewaddress("", "legacy")
alice_private_key_wif_format = alice.dumpprivkey(alice_address_1)
alice_private_key = PrivateKey.convert_wif_format(alice_private_key_wif_format)
alice_private_key_int_format = alice_private_key.get_private_key_int()
alice_public_key = alice_private_key.get_public_key()

alice_p2sh_funding_tx = alice.sendtoaddress(alice_address_1, 0.00011)
alice_p2sh_funding_tx_parsed = alice.decoderawtransaction(alice.gettransaction(alice_p2sh_funding_tx)["hex"])
test.generatetoaddress(alice, 1, alice_address_0)

bob_secret = "We make it visible"
bob_secret_hex = string_to_hex(bob_secret)
bob_original_script = Script([bytes.fromhex(bob_secret_hex), 0x87])
bob_original_script_hex = bob_original_script.serialize_script().hex()[2:]  
bob_original_script_hash160 = hash160(bytes.fromhex(bob_original_script_hex))

2024-03-03T11:53:59.780000Z TestFramework (INFO): PRNG seed is: 1856772094372594502
2024-03-03T11:53:59.780000Z TestFramework (INFO): Initializing test directory /tmp/bitcoin_func_test_325nw4jk


**1.1** In der ersten Transaktion werden von Alice die 10000 Satoshis an das P2SH Script von Bob versendet. Um das notwendige Transaktionsformular ausfüllen zu können, benötigt Alice folgende Informationen.

In [3]:
alice_previous_tx_id_to_spent = alice_p2sh_funding_tx_parsed["txid"]

for tx_out in alice_p2sh_funding_tx_parsed["vout"]:
    if tx_out["value"] == Decimal("0.00011000"):
        alice_previous_tx_index_to_spent = tx_out["n"]
        alice_previous_script_pub_key_to_spent = tx_out["scriptPubKey"]["hex"]

#Transaktionsoutputdaten
alice_amount_to_spent = 10000

**1.2** Nun kann Alice die Lockingtransaktion erzeugen. Um die Satoshis an Bob's Skript zu senden, muss Alice im Output der Transaktion ein scriptPubKey der Form: OP_HASH160 + <script_hash> + OP_EQUAL generieren.

In [4]:
transaction_input = CTxIn(bytes.fromhex(alice_previous_tx_id_to_spent), alice_previous_tx_index_to_spent, script_sig=Script())

alice_script_pubkey = p2sh_script(bob_original_script_hash160)
transaction_output = CTxOut(alice_amount_to_spent, alice_script_pubkey)

alice_locking_transaction = CTx(1, [transaction_input], [transaction_output], 0xffffffff, is_testnet=True, is_segwit=False)

print(f"Alice unsignierte P2SH Lockingtransaktion: {alice_locking_transaction.serialize_transaction().hex()}")

Alice unsignierte P2SH Lockingtransaktion: 010000000178f55195cbe36202914be544d6f34b64c4ba41fa1ec81a75208cbc65d0b9045b0000000000ffffffff01102700000000000017a914c7ccae441a968c81e8a49f92260473da4cd99d5587ffffffff


**1.3** Nun muss Alice die Lockingtransaktion noch signieren. D.h. sie muss nachweisen, dass sie aus der vorhergehenden Transaktion berechtigt ist die 10000 Satoshis auszugeben.

In [5]:
alice_script_sig = Script().parse_script(BytesIO(bytes.fromhex(f"{hex(len(alice_previous_script_pub_key_to_spent)//2)[2:]}{alice_previous_script_pub_key_to_spent}")))
alice_locking_transaction.sign_transaction(0, [alice_private_key_int_format], alice_script_sig)

print(f"Alice signierte P2SH Lockingtransaktion: {alice_locking_transaction.serialize_transaction().hex()}")

alice_p2sh_locking_tx = alice.sendrawtransaction(alice_locking_transaction.serialize_transaction().hex())
test.generatetoaddress(alice, 1, alice_address_0)

Alice signierte P2SH Lockingtransaktion: 010000000178f55195cbe36202914be544d6f34b64c4ba41fa1ec81a75208cbc65d0b9045b000000006a473044022058b009b617f0a9638162a934aad952bdaaf2cfdb55773896405908335648d6eb0220760c6daacee9dbe48d8dcca6a1f6b7facc541bba5885273333fa8ef6cd4dc15b012102a2448697bb52f8a21476af38de1ae9b0d7de59f83974c33a8e1a1b5bd5d8e355ffffffff01102700000000000017a914c7ccae441a968c81e8a49f92260473da4cd99d5587ffffffff
25c2e0e6fb86f185675a20f8fd3a3de8504aaf48192940a7af58329312f47fad


**2.** Nun möchte Bob die an sein Skript gebundenen Satoshis wieder ausgeben. In dem Fall muss er **keine** Signatur oder der gleichen erzeugen. Zum Beweis das er berechtigt ist die Funds auszugeben muss er im ScriptSig Feld des Inputs sein Secret und das Originalskript vorlegen.

In [6]:
alice_p2sh_locking_tx_parsed = alice.decoderawtransaction(alice.gettransaction(alice_p2sh_locking_tx)["hex"])

bob_previous_tx_id_to_spent = alice_p2sh_locking_tx_parsed["txid"]

for tx_out in alice_p2sh_locking_tx_parsed["vout"]:
    if tx_out["value"] == Decimal("0.00010000"):
        bob_previous_tx_index_to_spent = tx_out["n"]
        bob_previous_script_pub_key_to_spent = tx_out["scriptPubKey"]["hex"]

#Transaktionsoutputdaten
bob_amount_to_spent = 9000
#bob's reciever address is alice public key

**2.1** Nun kann Bob die Transaktion erstellen um die eben von Alice erhaltenen Satoshis weiter zu versenden.

In [7]:
script_sig = Script([bytes.fromhex(bob_secret_hex), bytes.fromhex(bob_original_script_hex)])
transaction_input = CTxIn(bytes.fromhex(bob_previous_tx_id_to_spent), bob_previous_tx_index_to_spent, script_sig=script_sig)

script_pubkey = p2pk_script(alice_public_key.sec_format())
transaction_output = CTxOut(bob_amount_to_spent, script_pubkey)

bob_spending_transaction = CTx(1, [transaction_input], [transaction_output], 0xffffffff, is_testnet=True, is_segwit=False)

print(f"Bob 'signierte' P2SH Spendingtransaktion: {bob_spending_transaction.serialize_transaction().hex()}")

TestShell().shutdown()

Bob 'signierte' P2SH Spendingtransaktion: 0100000001ad7ff412933258afa740291948af4a50e83d3afdf8205a6785f186fbe6e0c2250000000028125765206d616b652069742076697369626c6514125765206d616b652069742076697369626c6587ffffffff012823000000000000232102a2448697bb52f8a21476af38de1ae9b0d7de59f83974c33a8e1a1b5bd5d8e355acffffffff
2024-03-03T11:54:00.612000Z TestFramework (INFO): Stopping nodes
2024-03-03T11:54:00.715000Z TestFramework (INFO): Cleaning up /tmp/bitcoin_func_test_325nw4jk on exit
2024-03-03T11:54:00.715000Z TestFramework (INFO): Tests successful
